#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/competitions/desafio-de-prediccion-de-precios-de-portatiles

## Librerias

In [401]:
import bootcampviztools as bt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import recall_score, classification_report
from catboost import CatBoostRegressor
import lightgbm as lgb
import xgboost as xgb
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR


## Data exploration

In [207]:
ruta = r"D:\DATASCIENCE\Repositorios\Kaggle_Desafio\Train\train.csv"
train = pd.read_csv(ruta, index_col="id")

In [208]:
train_copia = train.copy()

In [209]:
train_copia.Cpu.head(30)

id
268               Intel Core i7 7500U 2.7GHz
347     Intel Pentium Quad Core N4200 1.1GHz
353               Intel Core i5 6200U 2.3GHz
578              Intel Core i7 7700HQ 2.8GHz
42                Intel Core i5 6300U 2.4GHz
867               Intel Core i7 7500U 2.7GHz
1136                Intel Core i3 6006U 2GHz
609              Intel Core i7 7700HQ 2.8GHz
474               Intel Core i7 7500U 2.7GHz
838              Intel Core i7 7700HQ 2.8GHz
851     Intel Pentium Quad Core N3710 1.6GHz
904               Intel Core i7 8550U 1.8GHz
355               Intel Core i5 6200U 2.3GHz
44                AMD A9-Series A9-9420 3GHz
36      Intel Pentium Dual Core 4405U 2.1GHz
971               Intel Core i5 8250U 1.6GHz
938               Intel Core i3 7100U 2.4GHz
718               Intel Core i5 7200U 2.5GHz
518            Intel Xeon E3-1535M v6 3.1GHz
1126              Intel Core i5 8250U 1.6GHz
51               Intel Atom x5-Z8550 1.44GHz
772              AMD A10-Series 9620P 2.5GHz
1001   

In [210]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 268 to 1179
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         912 non-null    int64  
 1   Company           912 non-null    object 
 2   Product           912 non-null    object 
 3   TypeName          912 non-null    object 
 4   Inches            912 non-null    float64
 5   ScreenResolution  912 non-null    object 
 6   Cpu               912 non-null    object 
 7   Ram               912 non-null    object 
 8   Memory            912 non-null    object 
 9   Gpu               912 non-null    object 
 10  OpSys             912 non-null    object 
 11  Weight            912 non-null    object 
 12  Price_euros       912 non-null    float64
dtypes: float64(2), int64(1), object(10)
memory usage: 99.8+ KB


In [211]:
train.TypeName.unique()

array(['Notebook', 'Gaming', 'Ultrabook', '2 in 1 Convertible', 'Netbook',
       'Workstation'], dtype=object)

In [212]:
train.Ram.unique()

array(['8GB', '4GB', '16GB', '12GB', '6GB', '32GB', '2GB', '24GB'],
      dtype=object)

In [213]:
train.Memory.unique()

array(['1TB HDD', '256GB SSD', '256GB SSD +  1TB HDD',
       '512GB SSD +  2TB HDD', '128GB SSD +  1TB HDD', '128GB SSD',
       '1TB SSD', '64GB Flash Storage', '2TB HDD', '500GB HDD',
       '1.0TB Hybrid', '16GB Flash Storage', '256GB Flash Storage',
       '180GB SSD', '32GB Flash Storage', '512GB SSD',
       '512GB SSD +  1TB HDD', '256GB SSD +  500GB HDD',
       '256GB SSD +  2TB HDD', '1TB SSD +  1TB HDD', '64GB SSD',
       '128GB Flash Storage', '256GB SSD +  1.0TB Hybrid', '32GB SSD',
       '512GB Flash Storage', '240GB SSD', '256GB SSD +  256GB SSD',
       '16GB SSD', '8GB SSD', '64GB Flash Storage +  1TB HDD',
       '1.0TB HDD', '1TB HDD +  1TB HDD', '512GB SSD +  512GB SSD',
       '128GB SSD +  2TB HDD', '32GB HDD', '512GB SSD +  1.0TB Hybrid'],
      dtype=object)

In [214]:
train.loc[train.TypeName == "Gaming"].max()

laptop_ID                                                    1277
Company                                                     Razer
Product                                              VivoBook Pro
TypeName                                                   Gaming
Inches                                                       18.4
ScreenResolution    IPS Panel Touchscreen / 4K Ultra HD 3840x2160
Cpu                                   Intel Core i7 7820HK 2.9GHz
Ram                                                           8GB
Memory                                       512GB SSD +  2TB HDD
Gpu                                     Nvidia GeForce GTX1050 Ti
OpSys                                                  Windows 10
Weight                                                        4kg
Price_euros                                                6099.0
dtype: object

In [215]:
train.loc[train.TypeName == "Notebook"].max()

laptop_ID                                    1320
Company                                    Xiaomi
Product                    Zenbook UX510UW-FI095T
TypeName                                 Notebook
Inches                                       17.3
ScreenResolution             Touchscreen 1366x768
Cpu                 Intel Xeon E3-1535M v6 3.1GHz
Ram                                           8GB
Memory                                   64GB SSD
Gpu                           Nvidia Quadro M620M
OpSys                                   Windows 7
Weight                                        3kg
Price_euros                                4899.0
dtype: object

In [216]:
train.loc[train.TypeName == "Gaming"].sort_values(by="Price_euros", ascending=True)

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
id,,,,,,,,,,,,,
131,92,Asus,FX550IK-DM018T (FX-9830P/8GB/1TB/Radeon,Gaming,15.6,Full HD 1920x1080,AMD FX 9830P 3GHz,8GB,1TB HDD,AMD Radeon RX 560,Windows 10,2.45kg,699.0
382,191,Lenovo,Legion Y520-15IKBN,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i5 7300HQ 2.5GHz,8GB,256GB SSD,Nvidia GeForce GTX 1050,No OS,2.4kg,779.0
227,373,Lenovo,Legion Y520-15IKBN,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i5 7300HQ 2.5GHz,8GB,128GB SSD + 1TB HDD,Nvidia GeForce GTX 1050,No OS,2.4kg,809.0
163,1208,Lenovo,Legion Y520-15IKBN,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i5 7300HQ 2.5GHz,8GB,1TB HDD,Nvidia GeForce GTX 1050,Windows 10,2.4kg,819.0
1270,708,Dell,Inspiron 5577,Gaming,15.6,Full HD 1920x1080,Intel Core i5 7300HQ 2.5GHz,8GB,1TB HDD,Nvidia GeForce GTX 1050,Windows 10,2.56kg,819.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167,1249,Razer,Blade Pro,Gaming,14.0,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,1TB SSD,Nvidia GeForce GTX 1060,Windows 10,1.95kg,3499.0
333,789,Dell,Alienware 17,Gaming,17.3,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,32GB,1TB SSD + 1TB HDD,Nvidia GeForce GTX 1070M,Windows 10,4.42kg,3588.8
813,731,Dell,Alienware 17,Gaming,17.3,4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,32GB,1TB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.36kg,3659.4


In [217]:
train.loc[train.TypeName == "Notebook"].sort_values(by="Price_euros", ascending=True)

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
id,,,,,,,,,,,,,
664,295,Acer,Chromebook C910-C2ST,Notebook,15.6,1366x768,Intel Celeron Dual Core 3205U 1.5GHz,2GB,16GB SSD,Intel HD Graphics,Chrome OS,2.19kg,199.0
1002,32,Asus,E402WA-GA010T (E2-6110/2GB/32GB/W10),Notebook,14.0,1366x768,AMD E-Series E2-6110 1.5GHz,2GB,32GB Flash Storage,AMD Radeon R2,Windows 10,1.65kg,199.0
526,800,Vero,K146 (N3350/4GB/32GB/W10),Notebook,14.0,1920x1080,Intel Celeron Dual Core N3350 1.1GHz,4GB,32GB Flash Storage,Intel HD Graphics 500,Windows 10,1.22kg,202.9
169,1117,Acer,Chromebook 15,Notebook,15.6,1366x768,Intel Celeron Dual Core 3205U 1.5GHz,4GB,16GB SSD,Intel HD Graphics,Chrome OS,2.20kg,209.0
1075,1055,Vero,V142 (X5-Z8350/2GB/32GB/W10),Notebook,14.0,1366x768,Intel Atom X5-Z8350 1.44GHz,2GB,32GB Flash Storage,Intel HD Graphics 400,Windows 10,1.45kg,210.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,1064,Lenovo,ThinkPad X1,Notebook,14.0,IPS Panel 2560x1440,Intel Core i7 6600U 2.6GHz,16GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.1kg,2620.0
220,335,Dell,XPS 15,Notebook,15.6,4K Ultra HD / Touchscreen 3840x2160,Intel Core i7 7700HQ 2.8GHz,32GB,1TB SSD,Nvidia GeForce GTX 1050,Windows 10,2.06kg,2639.0
720,1031,Lenovo,ThinkPad P70,Notebook,17.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 6820HQ 2.7GHz,16GB,512GB SSD,Nvidia Quadro 3000M,Windows 7,2.4kg,2968.0


In [218]:
train.head(10)

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
id,,,,,,,,,,,,,
268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00
347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200 1.1GHz,4GB,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00
353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00
578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99
42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37
867,376,Asus,ZenBook Flip,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,12GB,512GB SSD + 2TB HDD,Nvidia GeForce GT 940MX,Windows 10,2.26kg,1099.00
1136,60,Asus,X541UA-DM1897 (i3-6006U/4GB/256GB/FHD/Linux),Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,4GB,256GB SSD,Intel HD Graphics 520,Linux,2kg,415.00
609,1157,MSI,GP62M 7RDX,Gaming,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8GB,128GB SSD + 1TB HDD,Nvidia GeForce GTX 1050,Windows 10,2.4kg,1142.80
474,110,Asus,UX430UQ-GV209R (i7-7500U/8GB/256GB/GeForce,Ultrabook,14.0,IPS Panel Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,256GB SSD,Nvidia GeForce 940MX,Windows 10,1.3kg,1193.00


In [219]:
train.loc[train.Company == "Apple"]

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
id,,,,,,,,,,,,,
34,7,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.2GHz,16GB,256GB Flash Storage,Intel Iris Pro Graphics,Mac OS X,2.04kg,2139.97
291,83,Apple,"MacBook 12""",Ultrabook,12.0,IPS Panel Retina Display 2304x1440,Intel Core i5 1.3GHz,8GB,512GB SSD,Intel HD Graphics 615,macOS,0.92kg,1510.00
23,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
1192,254,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,512GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,2040.00
395,803,Apple,"MacBook 12""",Ultrabook,12.0,IPS Panel Retina Display 2304x1440,Intel Core M 1.2GHz,8GB,512GB Flash Storage,Intel HD Graphics 5300,Mac OS X,0.920kg,1165.00
427,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
112,8,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,256GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,1158.70
692,13,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.8GHz,16GB,256GB SSD,AMD Radeon Pro 555,macOS,1.83kg,2439.97
849,1211,Apple,"MacBook 12""",Ultrabook,12.0,IPS Panel Retina Display 2304x1440,Intel Core M 1.1GHz,8GB,256GB Flash Storage,Intel HD Graphics 5300,Mac OS X,0.920kg,1163.00


## Data processing

###LIMPIAR RESOLUCION

In [220]:
#EXTRACCION RESOLUCION DE PANTALLAS PARA LUEGO CLASIFICARLAS

patron = r'(\d{3,4}x\d{3,4})'  #Patron para extraer la resolucion solo

train["Resolucion"] = train['ScreenResolution'].str.extract(patron)

In [221]:
def clasificar_resolucion(valor):
    if valor == "640x480":
        return 'VGA'
    elif valor == "800x600":
        return 'SVGA'
    elif valor == "1024x768":
        return 'XGA'
    elif valor == "1366x768":
        return 'WXGA'
    elif valor == "1280x1024":
        return 'SXGA'
    elif valor == "1600x1200":
        return 'UXGA'
    elif valor == "1280x720":
        return 'HD'
    elif valor == "1920x1080":
        return 'FHD'
    elif valor == "2560x1440":
        return 'QHD'
    elif valor == "3840x2160":
        return '4K'
    elif valor == "5120x2880":
        return '5K'
    else:
        return 'IPS' 

In [222]:
train['Pantalla'] = train['Resolucion'].apply(clasificar_resolucion)

In [223]:
train.Pantalla.unique()

array(['WXGA', 'FHD', '4K', 'IPS', 'QHD'], dtype=object)

In [224]:
train.head(10)

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,Resolucion,Pantalla
id,,,,,,,,,,,,,,,
268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00,1366x768,WXGA
347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200 1.1GHz,4GB,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00,1366x768,WXGA
353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00,1920x1080,FHD
578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99,3840x2160,4K
42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37,1920x1080,FHD
867,376,Asus,ZenBook Flip,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,12GB,512GB SSD + 2TB HDD,Nvidia GeForce GT 940MX,Windows 10,2.26kg,1099.00,1920x1080,FHD
1136,60,Asus,X541UA-DM1897 (i3-6006U/4GB/256GB/FHD/Linux),Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,4GB,256GB SSD,Intel HD Graphics 520,Linux,2kg,415.00,1920x1080,FHD
609,1157,MSI,GP62M 7RDX,Gaming,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8GB,128GB SSD + 1TB HDD,Nvidia GeForce GTX 1050,Windows 10,2.4kg,1142.80,1920x1080,FHD
474,110,Asus,UX430UQ-GV209R (i7-7500U/8GB/256GB/GeForce,Ultrabook,14.0,IPS Panel Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,256GB SSD,Nvidia GeForce 940MX,Windows 10,1.3kg,1193.00,1920x1080,FHD


#EXTRACCION DE PROCESADOR


In [225]:
train.Cpu.unique()
#He sacado todo los valores del Cpu, para posteriormente ir con ayuda ir puliendo un codigo para que me genere la extracion de datos correctamente

array(['Intel Core i7 7500U 2.7GHz',
       'Intel Pentium Quad Core N4200 1.1GHz',
       'Intel Core i5 6200U 2.3GHz', 'Intel Core i7 7700HQ 2.8GHz',
       'Intel Core i5 6300U 2.4GHz', 'Intel Core i3 6006U 2GHz',
       'Intel Pentium Quad Core N3710 1.6GHz',
       'Intel Core i7 8550U 1.8GHz', 'AMD A9-Series A9-9420 3GHz',
       'Intel Pentium Dual Core 4405U 2.1GHz',
       'Intel Core i5 8250U 1.6GHz', 'Intel Core i3 7100U 2.4GHz',
       'Intel Core i5 7200U 2.5GHz', 'Intel Xeon E3-1535M v6 3.1GHz',
       'Intel Atom x5-Z8550 1.44GHz', 'AMD A10-Series 9620P 2.5GHz',
       'Intel Core i5 7300U 2.6GHz',
       'Intel Celeron Dual Core N3350 1.1GHz',
       'Intel Core i7 6500U 2.5GHz', 'Intel Core i3 6006U 2.0GHz',
       'Intel Celeron Dual Core 3205U 1.5GHz',
       'AMD A12-Series 9720P 3.6GHz',
       'Intel Celeron Dual Core N3050 1.6GHz',
       'Intel Core i3 7130U 2.7GHz', 'Intel Core i5 7300HQ 2.5GHz',
       'Intel Core i5 7200U 2.7GHz', 'Intel Core i7 7820HQ 2.9GHz

In [226]:
def extract_brand(cpu):
    return cpu.split()[0]

def extract_model(cpu):
    pattern = re.compile(r'(Core [iM][3579]|Core M \w+|Pentium Quad Core|Pentium Dual Core|Celeron Quad Core|Celeron Dual Core|Atom \w+|Xeon \w+-\w+|FX \d+|Ryzen \d+|A\d+-Series|E-Series)')
    match = pattern.search(cpu)
    return match.group(0) if match else "Otro"

def extract_speed(cpu):
    pattern = re.compile(r'(\d+\.\d+GHz|\d+GHz)')
    match = pattern.search(cpu)
    return match.group(0) if match else "Otro"



In [227]:
train['Marca_Procesador'] = train['Cpu'].apply(extract_brand)
train['Modelo_Procesador'] = train['Cpu'].apply(extract_model)
train['Velocidad_Procesador'] = train['Cpu'].apply(extract_speed)




In [228]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 268 to 1179
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   laptop_ID             912 non-null    int64  
 1   Company               912 non-null    object 
 2   Product               912 non-null    object 
 3   TypeName              912 non-null    object 
 4   Inches                912 non-null    float64
 5   ScreenResolution      912 non-null    object 
 6   Cpu                   912 non-null    object 
 7   Ram                   912 non-null    object 
 8   Memory                912 non-null    object 
 9   Gpu                   912 non-null    object 
 10  OpSys                 912 non-null    object 
 11  Weight                912 non-null    object 
 12  Price_euros           912 non-null    float64
 13  Resolucion            912 non-null    object 
 14  Pantalla              912 non-null    object 
 15  Marca_Procesador      912

#EXTRACCION DE DISCOS DUROS

In [229]:
train.Memory.unique()

array(['1TB HDD', '256GB SSD', '256GB SSD +  1TB HDD',
       '512GB SSD +  2TB HDD', '128GB SSD +  1TB HDD', '128GB SSD',
       '1TB SSD', '64GB Flash Storage', '2TB HDD', '500GB HDD',
       '1.0TB Hybrid', '16GB Flash Storage', '256GB Flash Storage',
       '180GB SSD', '32GB Flash Storage', '512GB SSD',
       '512GB SSD +  1TB HDD', '256GB SSD +  500GB HDD',
       '256GB SSD +  2TB HDD', '1TB SSD +  1TB HDD', '64GB SSD',
       '128GB Flash Storage', '256GB SSD +  1.0TB Hybrid', '32GB SSD',
       '512GB Flash Storage', '240GB SSD', '256GB SSD +  256GB SSD',
       '16GB SSD', '8GB SSD', '64GB Flash Storage +  1TB HDD',
       '1.0TB HDD', '1TB HDD +  1TB HDD', '512GB SSD +  512GB SSD',
       '128GB SSD +  2TB HDD', '32GB HDD', '512GB SSD +  1.0TB Hybrid'],
      dtype=object)

In [230]:
def separar_discos(disco):
    # Inicializa las capacidades
    capacidad_ssd = '0 GB'
    capacidad_hdd = '0 GB'
    capacidad_flash = '0 GB'
    capacidad_hybrid = '0 GB'

    # Busca la capacidad y tipo de disco en la cadena
    matches = re.findall(r'(\d+(\.\d+)?)(GB|TB)\s+(SSD|HDD|Flash Storage|Hybrid)', disco)


    for capacidad, _, unidad, tipo in matches:
        capacidad_completa = f"{capacidad} {unidad}"
        if tipo == 'SSD':
            capacidad_ssd = capacidad_completa
        elif tipo == 'HDD':
            capacidad_hdd = capacidad_completa
        elif tipo == 'Flash Storage':
            capacidad_flash = capacidad_completa
        elif tipo == 'Hybrid':
            capacidad_hybrid = capacidad_completa

    return capacidad_ssd, capacidad_hdd, capacidad_flash, capacidad_hybrid

In [231]:
train[['SSD', 'HDD', 'Flash Storage', 'Hybrid']] = train['Memory'].apply(lambda x: pd.Series(separar_discos(x)))

In [232]:
train.head(10)

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,...,Price_euros,Resolucion,Pantalla,Marca_Procesador,Modelo_Procesador,Velocidad_Procesador,SSD,HDD,Flash Storage,Hybrid
id,,,,,,,,,,,,,,,,,,,,,
268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R7 M445,...,749.00,1366x768,WXGA,Intel,Core i7,2.7GHz,0 GB,1 TB,0 GB,0 GB
347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200 1.1GHz,4GB,1TB HDD,Intel HD Graphics 505,...,449.00,1366x768,WXGA,Intel,Pentium Quad Core,1.1GHz,0 GB,1 TB,0 GB,0 GB
353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,...,1460.00,1920x1080,FHD,Intel,Core i5,2.3GHz,256 GB,0 GB,0 GB,0 GB
578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,...,2868.99,3840x2160,4K,Intel,Core i7,2.8GHz,256 GB,1 TB,0 GB,0 GB
42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,...,1713.37,1920x1080,FHD,Intel,Core i5,2.4GHz,256 GB,0 GB,0 GB,0 GB
867,376,Asus,ZenBook Flip,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,12GB,512GB SSD + 2TB HDD,Nvidia GeForce GT 940MX,...,1099.00,1920x1080,FHD,Intel,Core i7,2.7GHz,512 GB,2 TB,0 GB,0 GB
1136,60,Asus,X541UA-DM1897 (i3-6006U/4GB/256GB/FHD/Linux),Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,4GB,256GB SSD,Intel HD Graphics 520,...,415.00,1920x1080,FHD,Intel,Core i3,2GHz,256 GB,0 GB,0 GB,0 GB
609,1157,MSI,GP62M 7RDX,Gaming,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8GB,128GB SSD + 1TB HDD,Nvidia GeForce GTX 1050,...,1142.80,1920x1080,FHD,Intel,Core i7,2.8GHz,128 GB,1 TB,0 GB,0 GB
474,110,Asus,UX430UQ-GV209R (i7-7500U/8GB/256GB/GeForce,Ultrabook,14.0,IPS Panel Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,256GB SSD,Nvidia GeForce 940MX,...,1193.00,1920x1080,FHD,Intel,Core i7,2.7GHz,256 GB,0 GB,0 GB,0 GB


#EXTRACCION KG

In [233]:
def extraer_numerico(texto):
    """
    Esta función extrae el primer valor numérico (entero o decimal) encontrado en una cadena de texto.
    Si no se encuentra un número, devuelve None.
    """
    coincidencias = re.findall(r'\d+\.?\d*', texto)
    if coincidencias:
        return float(coincidencias[0])
    return None

In [234]:
train['Peso_KG'] = train['Weight'].apply(extraer_numerico)

In [235]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 268 to 1179
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   laptop_ID             912 non-null    int64  
 1   Company               912 non-null    object 
 2   Product               912 non-null    object 
 3   TypeName              912 non-null    object 
 4   Inches                912 non-null    float64
 5   ScreenResolution      912 non-null    object 
 6   Cpu                   912 non-null    object 
 7   Ram                   912 non-null    object 
 8   Memory                912 non-null    object 
 9   Gpu                   912 non-null    object 
 10  OpSys                 912 non-null    object 
 11  Weight                912 non-null    object 
 12  Price_euros           912 non-null    float64
 13  Resolucion            912 non-null    object 
 14  Pantalla              912 non-null    object 
 15  Marca_Procesador      912

###AQUI SEPARAMOS YA CON UNA PRIMERA LIMPIEZA DE DATOS PARA VER MEJOR LO QUE TENEMOS PARA HACER UNA SEGUNDA LIMPIEZA 

In [236]:
train_limpio_1 = train.drop(columns=["Inches","Weight","ScreenResolution","Cpu","Memory","Resolucion"])

In [237]:
train_limpio_1.to_csv(r"D:\DATASCIENCE\Repositorios\Kaggle_Desafio\Train\TrainLimpio1.csv", index=True)

In [238]:
train_limpio_1

,laptop_ID,Company,Product,TypeName,Ram,Gpu,OpSys,Price_euros,Pantalla,Marca_Procesador,Modelo_Procesador,Velocidad_Procesador,SSD,HDD,Flash Storage,Hybrid,Peso_KG
id,,,,,,,,,,,,,,,,,
268,1002,Dell,Inspiron 5567,Notebook,8GB,AMD Radeon R7 M445,Windows 10,749.00,WXGA,Intel,Core i7,2.7GHz,0 GB,1 TB,0 GB,0 GB,2.36
347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,4GB,Intel HD Graphics 505,Windows 10,449.00,WXGA,Intel,Pentium Quad Core,1.1GHz,0 GB,1 TB,0 GB,0 GB,2.00
353,966,Toshiba,Portege Z30-C-1CW,Notebook,8GB,Intel HD Graphics 520,Windows 7,1460.00,FHD,Intel,Core i5,2.3GHz,256 GB,0 GB,0 GB,0 GB,1.20
578,767,Dell,Alienware 17,Gaming,16GB,Nvidia GeForce GTX 1070,Windows 10,2868.99,4K,Intel,Core i7,2.8GHz,256 GB,1 TB,0 GB,0 GB,4.42
42,1241,Dell,Latitude E7270,Ultrabook,8GB,Intel HD Graphics 520,Windows 7,1713.37,FHD,Intel,Core i5,2.4GHz,256 GB,0 GB,0 GB,0 GB,1.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,973,Acer,Chromebook C738T-C2EJ,2 in 1 Convertible,4GB,Intel HD Graphics 400,Chrome OS,389.00,WXGA,Intel,Celeron Dual Core,1.6GHz,0 GB,0 GB,32 GB,0 GB,1.25
716,1018,Toshiba,Portege Z30-C-16H,Notebook,4GB,Intel HD Graphics 520,Windows 10,1195.00,FHD,Intel,Core i5,2.3GHz,128 GB,0 GB,0 GB,0 GB,1.20
517,1071,Acer,Aspire ES1-523,Notebook,8GB,AMD Radeon R5,Windows 10,449.00,WXGA,AMD,A8-Series,2.2GHz,0 GB,1 TB,0 GB,0 GB,2.40


###SEGUNDA LIMPIEZA DONDE VAMOS A:

**RAM**: Quedarnos con el numerico solo, y transformalor a INT. Cambio nombre columna a RAM_GB <br>
**OPSys**: Cambiar el nombre de la columna a S.O <br>
**Velocidad_Procesador**: Quedarnos con el númerico y transformarlo a Float. Cambio nombre a Velocidad_Procesador_GHz <br>
**Peso_Kg**: Definir tres categorias a partir del peso. Ligero, Medio, Pesado.

**Peso_Kg**: Definir tres categorias a partir del peso. Ligero, Medio, Pesado.

In [239]:
def categorize_weight(weight):
    if weight < 1.5:
        return 'Ligero'
    elif 1.5 <= weight <= 2.5:
        return 'Medio'
    else:
        return 'Pesado'


In [240]:
train_limpio_1['Categoria_Peso'] = train_limpio_1['Peso_KG'].apply(categorize_weight)

In [241]:
train_limpio_1.head(5)

,laptop_ID,Company,Product,TypeName,Ram,Gpu,OpSys,Price_euros,Pantalla,Marca_Procesador,Modelo_Procesador,Velocidad_Procesador,SSD,HDD,Flash Storage,Hybrid,Peso_KG,Categoria_Peso
id,,,,,,,,,,,,,,,,,,
268,1002,Dell,Inspiron 5567,Notebook,8GB,AMD Radeon R7 M445,Windows 10,749.00,WXGA,Intel,Core i7,2.7GHz,0 GB,1 TB,0 GB,0 GB,2.36,Medio
347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,4GB,Intel HD Graphics 505,Windows 10,449.00,WXGA,Intel,Pentium Quad Core,1.1GHz,0 GB,1 TB,0 GB,0 GB,2.00,Medio
353,966,Toshiba,Portege Z30-C-1CW,Notebook,8GB,Intel HD Graphics 520,Windows 7,1460.00,FHD,Intel,Core i5,2.3GHz,256 GB,0 GB,0 GB,0 GB,1.20,Ligero
578,767,Dell,Alienware 17,Gaming,16GB,Nvidia GeForce GTX 1070,Windows 10,2868.99,4K,Intel,Core i7,2.8GHz,256 GB,1 TB,0 GB,0 GB,4.42,Pesado
42,1241,Dell,Latitude E7270,Ultrabook,8GB,Intel HD Graphics 520,Windows 7,1713.37,FHD,Intel,Core i5,2.4GHz,256 GB,0 GB,0 GB,0 GB,1.26,Ligero


**Velocidad_Procesador**: Quedarnos con el númerico y transformarlo a Float. Cambio nombre a Velocidad_Procesador_GHz

In [242]:
train_limpio_1['Velocidad_Procesador_GHz'] = train_limpio_1['Velocidad_Procesador'].str.extract(r'(\d+\.?\d*)').astype(float)

In [243]:
train_limpio_1

,laptop_ID,Company,Product,TypeName,Ram,Gpu,OpSys,Price_euros,Pantalla,Marca_Procesador,Modelo_Procesador,Velocidad_Procesador,SSD,HDD,Flash Storage,Hybrid,Peso_KG,Categoria_Peso,Velocidad_Procesador_GHz
id,,,,,,,,,,,,,,,,,,,
268,1002,Dell,Inspiron 5567,Notebook,8GB,AMD Radeon R7 M445,Windows 10,749.00,WXGA,Intel,Core i7,2.7GHz,0 GB,1 TB,0 GB,0 GB,2.36,Medio,2.7
347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,4GB,Intel HD Graphics 505,Windows 10,449.00,WXGA,Intel,Pentium Quad Core,1.1GHz,0 GB,1 TB,0 GB,0 GB,2.00,Medio,1.1
353,966,Toshiba,Portege Z30-C-1CW,Notebook,8GB,Intel HD Graphics 520,Windows 7,1460.00,FHD,Intel,Core i5,2.3GHz,256 GB,0 GB,0 GB,0 GB,1.20,Ligero,2.3
578,767,Dell,Alienware 17,Gaming,16GB,Nvidia GeForce GTX 1070,Windows 10,2868.99,4K,Intel,Core i7,2.8GHz,256 GB,1 TB,0 GB,0 GB,4.42,Pesado,2.8
42,1241,Dell,Latitude E7270,Ultrabook,8GB,Intel HD Graphics 520,Windows 7,1713.37,FHD,Intel,Core i5,2.4GHz,256 GB,0 GB,0 GB,0 GB,1.26,Ligero,2.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,973,Acer,Chromebook C738T-C2EJ,2 in 1 Convertible,4GB,Intel HD Graphics 400,Chrome OS,389.00,WXGA,Intel,Celeron Dual Core,1.6GHz,0 GB,0 GB,32 GB,0 GB,1.25,Ligero,1.6
716,1018,Toshiba,Portege Z30-C-16H,Notebook,4GB,Intel HD Graphics 520,Windows 10,1195.00,FHD,Intel,Core i5,2.3GHz,128 GB,0 GB,0 GB,0 GB,1.20,Ligero,2.3
517,1071,Acer,Aspire ES1-523,Notebook,8GB,AMD Radeon R5,Windows 10,449.00,WXGA,AMD,A8-Series,2.2GHz,0 GB,1 TB,0 GB,0 GB,2.40,Medio,2.2


**OPSys**: Cambiar el nombre de la columna a S.O

In [244]:
train_limpio_1["S.O"] = train_limpio_1["OpSys"]

In [245]:
train_limpio_1["S.O"].unique()

array(['Windows 10', 'Windows 7', 'Linux', 'Chrome OS', 'No OS',
       'Mac OS X', 'Windows 10 S', 'macOS', 'Android'], dtype=object)

In [246]:
train_limpio_1.head(5)

,laptop_ID,Company,Product,TypeName,Ram,Gpu,OpSys,Price_euros,Pantalla,Marca_Procesador,Modelo_Procesador,Velocidad_Procesador,SSD,HDD,Flash Storage,Hybrid,Peso_KG,Categoria_Peso,Velocidad_Procesador_GHz,S.O
id,,,,,,,,,,,,,,,,,,,,
268,1002,Dell,Inspiron 5567,Notebook,8GB,AMD Radeon R7 M445,Windows 10,749.00,WXGA,Intel,Core i7,2.7GHz,0 GB,1 TB,0 GB,0 GB,2.36,Medio,2.7,Windows 10
347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,4GB,Intel HD Graphics 505,Windows 10,449.00,WXGA,Intel,Pentium Quad Core,1.1GHz,0 GB,1 TB,0 GB,0 GB,2.00,Medio,1.1,Windows 10
353,966,Toshiba,Portege Z30-C-1CW,Notebook,8GB,Intel HD Graphics 520,Windows 7,1460.00,FHD,Intel,Core i5,2.3GHz,256 GB,0 GB,0 GB,0 GB,1.20,Ligero,2.3,Windows 7
578,767,Dell,Alienware 17,Gaming,16GB,Nvidia GeForce GTX 1070,Windows 10,2868.99,4K,Intel,Core i7,2.8GHz,256 GB,1 TB,0 GB,0 GB,4.42,Pesado,2.8,Windows 10
42,1241,Dell,Latitude E7270,Ultrabook,8GB,Intel HD Graphics 520,Windows 7,1713.37,FHD,Intel,Core i5,2.4GHz,256 GB,0 GB,0 GB,0 GB,1.26,Ligero,2.4,Windows 7


**RAM**: Quedarnos con el numerico solo, y transformalor a INT. Cambio nombre columna a RAM_GB

In [247]:
train_limpio_1['Ram_GB'] = train_limpio_1['Ram'].str.extract(r'(\d+)\s*').astype(int)

In [248]:
train_limpio_1.head(3)

,laptop_ID,Company,Product,TypeName,Ram,Gpu,OpSys,Price_euros,Pantalla,Marca_Procesador,...,Velocidad_Procesador,SSD,HDD,Flash Storage,Hybrid,Peso_KG,Categoria_Peso,Velocidad_Procesador_GHz,S.O,Ram_GB
id,,,,,,,,,,,,,,,,,,,,,
268,1002,Dell,Inspiron 5567,Notebook,8GB,AMD Radeon R7 M445,Windows 10,749.0,WXGA,Intel,...,2.7GHz,0 GB,1 TB,0 GB,0 GB,2.36,Medio,2.7,Windows 10,8
347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,4GB,Intel HD Graphics 505,Windows 10,449.0,WXGA,Intel,...,1.1GHz,0 GB,1 TB,0 GB,0 GB,2.00,Medio,1.1,Windows 10,4
353,966,Toshiba,Portege Z30-C-1CW,Notebook,8GB,Intel HD Graphics 520,Windows 7,1460.0,FHD,Intel,...,2.3GHz,256 GB,0 GB,0 GB,0 GB,1.20,Ligero,2.3,Windows 7,8


In [249]:
train_limpio_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 268 to 1179
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   laptop_ID                 912 non-null    int64  
 1   Company                   912 non-null    object 
 2   Product                   912 non-null    object 
 3   TypeName                  912 non-null    object 
 4   Ram                       912 non-null    object 
 5   Gpu                       912 non-null    object 
 6   OpSys                     912 non-null    object 
 7   Price_euros               912 non-null    float64
 8   Pantalla                  912 non-null    object 
 9   Marca_Procesador          912 non-null    object 
 10  Modelo_Procesador         912 non-null    object 
 11  Velocidad_Procesador      912 non-null    object 
 12  SSD                       912 non-null    object 
 13  HDD                       912 non-null    object 
 14  Flash Storag

####Vamos a guardar esta segunda limpieza en otro csv. Asi lo tenemos los pasos mas claros y seguros para ir poco a poco modificando el dataframe

In [250]:
train_limpio_2 = train_limpio_1.drop(columns=["Ram","OpSys","Velocidad_Procesador","Peso_KG"])

In [251]:
train_limpio_2.to_csv(r"D:\DATASCIENCE\Repositorios\Kaggle_Desafio\Train\TrainLimpio2.csv", index=True)

In [252]:
train_limpio_2.head(2)

,laptop_ID,Company,Product,TypeName,Gpu,Price_euros,Pantalla,Marca_Procesador,Modelo_Procesador,SSD,HDD,Flash Storage,Hybrid,Categoria_Peso,Velocidad_Procesador_GHz,S.O,Ram_GB
id,,,,,,,,,,,,,,,,,
268,1002,Dell,Inspiron 5567,Notebook,AMD Radeon R7 M445,749.0,WXGA,Intel,Core i7,0 GB,1 TB,0 GB,0 GB,Medio,2.7,Windows 10,8
347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,Intel HD Graphics 505,449.0,WXGA,Intel,Pentium Quad Core,0 GB,1 TB,0 GB,0 GB,Medio,1.1,Windows 10,4


###TERCERA LIMPIEZA DONDE VAMOS A:

**TypeName**: Limpiar el 2 in 1 de Convertibles y cambiar nombre de la columna a Tipo_Portatil <br>
**Product**: Ver si podemos hacer alguna limpieza <br>



**TypeName**: Limpiar el 2 in 1 de Convertibles y cambiar nombre de la columna a Tipo_Portatil <br>

In [253]:
train_limpio_2['Tipo_Portatil'] = train_limpio_2['TypeName'].str.replace('2 in 1', '').str.strip()

In [254]:
train_limpio_2.Tipo_Portatil.unique()

array(['Notebook', 'Gaming', 'Ultrabook', 'Convertible', 'Netbook',
       'Workstation'], dtype=object)

In [255]:
train_limpio_2.head(2)

,laptop_ID,Company,Product,TypeName,Gpu,Price_euros,Pantalla,Marca_Procesador,Modelo_Procesador,SSD,HDD,Flash Storage,Hybrid,Categoria_Peso,Velocidad_Procesador_GHz,S.O,Ram_GB,Tipo_Portatil
id,,,,,,,,,,,,,,,,,,
268,1002,Dell,Inspiron 5567,Notebook,AMD Radeon R7 M445,749.0,WXGA,Intel,Core i7,0 GB,1 TB,0 GB,0 GB,Medio,2.7,Windows 10,8,Notebook
347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,Intel HD Graphics 505,449.0,WXGA,Intel,Pentium Quad Core,0 GB,1 TB,0 GB,0 GB,Medio,1.1,Windows 10,4,Notebook


**Product**: Ver si podemos hacer alguna limpieza <br>

In [256]:
def limpieza_producto(product_name):
    return re.split(r'\s*[\(\[]', product_name)[0].strip()

#Hemos limpiado lo que vemos entre parentesis de los nombres de los productos

In [257]:
train_limpio_2['Nombre_Producto'] = train_limpio_2['Product'].apply(limpieza_producto)

In [258]:
train_limpio_2.Nombre_Producto.head(5)

id
268        Inspiron 5567
347               X541NA
353    Portege Z30-C-1CW
578         Alienware 17
42        Latitude E7270
Name: Nombre_Producto, dtype: object

In [259]:
train_limpio_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 268 to 1179
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   laptop_ID                 912 non-null    int64  
 1   Company                   912 non-null    object 
 2   Product                   912 non-null    object 
 3   TypeName                  912 non-null    object 
 4   Gpu                       912 non-null    object 
 5   Price_euros               912 non-null    float64
 6   Pantalla                  912 non-null    object 
 7   Marca_Procesador          912 non-null    object 
 8   Modelo_Procesador         912 non-null    object 
 9   SSD                       912 non-null    object 
 10  HDD                       912 non-null    object 
 11  Flash Storage             912 non-null    object 
 12  Hybrid                    912 non-null    object 
 13  Categoria_Peso            912 non-null    object 
 14  Velocidad_Pr

####Vamos a guardar esta tercera limpieza en otro csv. 

In [260]:
train_limpio_3 = train_limpio_2.drop(columns=["TypeName","Product"])

In [261]:
train_limpio_3.to_csv(r"D:\DATASCIENCE\Repositorios\Kaggle_Desafio\Train\TrainLimpio3.csv", index=True)

In [262]:
train_limpio_3

,laptop_ID,Company,Gpu,Price_euros,Pantalla,Marca_Procesador,Modelo_Procesador,SSD,HDD,Flash Storage,Hybrid,Categoria_Peso,Velocidad_Procesador_GHz,S.O,Ram_GB,Tipo_Portatil,Nombre_Producto
id,,,,,,,,,,,,,,,,,
268,1002,Dell,AMD Radeon R7 M445,749.00,WXGA,Intel,Core i7,0 GB,1 TB,0 GB,0 GB,Medio,2.7,Windows 10,8,Notebook,Inspiron 5567
347,867,Asus,Intel HD Graphics 505,449.00,WXGA,Intel,Pentium Quad Core,0 GB,1 TB,0 GB,0 GB,Medio,1.1,Windows 10,4,Notebook,X541NA
353,966,Toshiba,Intel HD Graphics 520,1460.00,FHD,Intel,Core i5,256 GB,0 GB,0 GB,0 GB,Ligero,2.3,Windows 7,8,Notebook,Portege Z30-C-1CW
578,767,Dell,Nvidia GeForce GTX 1070,2868.99,4K,Intel,Core i7,256 GB,1 TB,0 GB,0 GB,Pesado,2.8,Windows 10,16,Gaming,Alienware 17
42,1241,Dell,Intel HD Graphics 520,1713.37,FHD,Intel,Core i5,256 GB,0 GB,0 GB,0 GB,Ligero,2.4,Windows 7,8,Ultrabook,Latitude E7270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,973,Acer,Intel HD Graphics 400,389.00,WXGA,Intel,Celeron Dual Core,0 GB,0 GB,32 GB,0 GB,Ligero,1.6,Chrome OS,4,Convertible,Chromebook C738T-C2EJ
716,1018,Toshiba,Intel HD Graphics 520,1195.00,FHD,Intel,Core i5,128 GB,0 GB,0 GB,0 GB,Ligero,2.3,Windows 10,4,Notebook,Portege Z30-C-16H
517,1071,Acer,AMD Radeon R5,449.00,WXGA,AMD,A8-Series,0 GB,1 TB,0 GB,0 GB,Medio,2.2,Windows 10,8,Notebook,Aspire ES1-523


####INTENTAMOS DARLE PESO DE ALGUNA MANERA A CIERTOS CAMPOS QUE TIENEN MAS PESO QUE OTROS.

In [263]:
#Funcion para quitar TB Y GB de las columnas y transformar TB a GB y tener todo en la misma proporcion
def convert_storage(value):
    if 'TB' in value:
        return float(value.replace(' TB', '')) * 1024
    elif 'GB' in value:
        return float(value.replace(' GB', ''))
    else:
        return 0.0


In [264]:
#Vamos a pasar por la columnas que queremos aplicar esta limpieza y transformación
train_limpio_3['SSD'] = train_limpio_3['SSD'].apply(convert_storage)
train_limpio_3['HDD'] = train_limpio_3['HDD'].apply(convert_storage)
train_limpio_3['Flash Storage'] = train_limpio_3['Flash Storage'].apply(convert_storage)
train_limpio_3['Hybrid'] = train_limpio_3['Hybrid'].apply(convert_storage)


In [265]:
#Como sabemos los discos SSD son los mas caros comparados con los otros y dependiendo de la capaciad de almacenamiento varia el precio.
#Vamos crear un campo nuevo con total almacenamiento dandole pesos diferentes según el tipo de disco.

train_limpio_3['Total_Almacenamiento'] = (train_limpio_3['SSD'] * 4) + (train_limpio_3['Hybrid'] * 3) + (train_limpio_3['HDD'] * 2) + train_limpio_3['Flash Storage']

In [266]:
train_limpio_3.Company.unique()

array(['Dell', 'Asus', 'Toshiba', 'MSI', 'HP', 'Lenovo', 'Acer', 'Apple',
       'Fujitsu', 'Mediacom', 'Razer', 'Samsung', 'LG', 'Microsoft',
       'Vero', 'Google', 'Huawei', 'Xiaomi', 'Chuwi'], dtype=object)

In [267]:
train_limpio_3.Tipo_Portatil.unique()

array(['Notebook', 'Gaming', 'Ultrabook', 'Convertible', 'Netbook',
       'Workstation'], dtype=object)

In [268]:
procesador = train_limpio_3.Velocidad_Procesador_GHz.unique()
procesador

array([2.7 , 1.1 , 2.3 , 2.8 , 2.4 , 2.  , 1.6 , 1.8 , 3.  , 2.1 , 2.5 ,
       3.1 , 1.44, 2.6 , 1.5 , 3.6 , 2.9 , 2.2 , 1.2 , 1.  , 1.3 , 3.2 ,
       0.9 , 1.9 , 1.92])

In [269]:
train_limpio_3.Modelo_Procesador.unique()

array(['Core i7', 'Pentium Quad Core', 'Core i5', 'Core i3', 'A9-Series',
       'Pentium Dual Core', 'Xeon E3-1535M', 'Atom x5', 'A10-Series',
       'Celeron Dual Core', 'A12-Series', 'Core M 6Y75', 'E-Series',
       'FX 8800', 'Core M 7Y30', 'A6-Series', 'Ryzen 1600',
       'Celeron Quad Core', 'Core M M3', 'Core M 6Y30', 'Core M 1',
       'A8-Series', 'Xeon E3-1505M', 'Ryzen 1700', 'Core M M7',
       'Core M m3', 'Atom X5', 'FX 9830', 'Atom Z8350'], dtype=object)

In [270]:
#Vemos que CORE M M3 esta en mayusculas y minusculas

train_limpio_3['Modelo_Procesador'] = train_limpio_3['Modelo_Procesador'].replace('Core M m3', 'Core M M3')
train_limpio_3['Modelo_Procesador'] = train_limpio_3['Modelo_Procesador'].replace('Core M 1', 'Core M M1')


In [271]:
train_limpio_3.Modelo_Procesador.unique()

array(['Core i7', 'Pentium Quad Core', 'Core i5', 'Core i3', 'A9-Series',
       'Pentium Dual Core', 'Xeon E3-1535M', 'Atom x5', 'A10-Series',
       'Celeron Dual Core', 'A12-Series', 'Core M 6Y75', 'E-Series',
       'FX 8800', 'Core M 7Y30', 'A6-Series', 'Ryzen 1600',
       'Celeron Quad Core', 'Core M M3', 'Core M 6Y30', 'Core M M1',
       'A8-Series', 'Xeon E3-1505M', 'Ryzen 1700', 'Core M M7', 'Atom X5',
       'FX 9830', 'Atom Z8350'], dtype=object)

In [272]:
train_limpio_3.Pantalla.unique()

array(['WXGA', 'FHD', '4K', 'IPS', 'QHD'], dtype=object)

In [273]:
train_limpio_3["S.O"].unique()

array(['Windows 10', 'Windows 7', 'Linux', 'Chrome OS', 'No OS',
       'Mac OS X', 'Windows 10 S', 'macOS', 'Android'], dtype=object)

In [274]:
train_limpio_3.Total_Almacenamiento.unique()

array([2048., 1024., 3072., 6144., 2560.,  512., 4096.,   64., 1000.,
         16.,  256.,  720.,   32., 2024., 5120.,  128.,  960., 2112.,
       4608.])

In [275]:
train_limpio_3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 268 to 1179
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   laptop_ID                 912 non-null    int64  
 1   Company                   912 non-null    object 
 2   Gpu                       912 non-null    object 
 3   Price_euros               912 non-null    float64
 4   Pantalla                  912 non-null    object 
 5   Marca_Procesador          912 non-null    object 
 6   Modelo_Procesador         912 non-null    object 
 7   SSD                       912 non-null    float64
 8   HDD                       912 non-null    float64
 9   Flash Storage             912 non-null    float64
 10  Hybrid                    912 non-null    float64
 11  Categoria_Peso            912 non-null    object 
 12  Velocidad_Procesador_GHz  912 non-null    float64
 13  S.O                       912 non-null    object 
 14  Ram_GB      

In [276]:
train_limpio_4 = train_limpio_3.drop(columns=["SSD","HDD","Flash Storage","Hybrid"])

In [277]:
train_limpio_4.to_csv(r"D:\DATASCIENCE\Repositorios\Kaggle_Desafio\Train\TrainLimpio4.csv", index=True)

###CUARTA LIMPIEZA DONDE VAMOS A:

**GPU**: Separar marca con modelo y generacion de dos columnas nuevas <br>


In [278]:
train_limpio_4.Gpu.unique()

array(['AMD Radeon R7 M445', 'Intel HD Graphics 505',
       'Intel HD Graphics 520', 'Nvidia GeForce GTX 1070',
       'Nvidia GeForce GT 940MX', 'Nvidia GeForce GTX 1050',
       'Nvidia GeForce 940MX', 'Nvidia GeForce GTX 1050 Ti',
       'Intel HD Graphics 405', 'Intel UHD Graphics 620',
       'AMD Radeon 530', 'Intel HD Graphics 510', 'Intel HD Graphics 620',
       'Nvidia Quadro M2200M', 'Nvidia GeForce MX150',
       'Intel HD Graphics 400', 'Nvidia Quadro M1200',
       'Nvidia GeForce 920MX', 'Intel HD Graphics 500', 'AMD Radeon 520',
       'AMD Radeon R5 M330', 'AMD Radeon R5 M420X', 'Intel HD Graphics',
       'Intel HD Graphics 630', 'AMD Radeon R5 M430',
       'Intel Iris Pro Graphics', 'Nvidia GeForce GTX 940MX',
       'Intel HD Graphics 515', 'Nvidia GeForce GTX 1080',
       'AMD Radeon R2 Graphics', 'Nvidia GeForce 150MX', 'AMD Radeon R5',
       'Nvidia GeForce GTX 1050Ti', 'AMD Radeon R9 M385',
       'Nvidia GeForce 930MX', 'Nvidia GeForce GTX 1060',
       'Nv

In [279]:
def separar_marca_modelo(gpu):
    # Extraer la marca
    marca_match = re.match(r'^(AMD|Intel|Nvidia)', gpu)
    marca = marca_match.group(0) if marca_match else 'Unknown'

    # Extraer el modelo
    modelo = gpu.replace(marca, '').strip()
    
    return pd.Series([marca, modelo], index=['Marca', 'Modelo'])

In [280]:
train_limpio_4[['Marca_GPU', 'Modelo_GPU']] = train_limpio_4['Gpu'].apply(separar_marca_modelo)

In [281]:
train_limpio_4.Marca_GPU.unique()

array(['AMD', 'Intel', 'Nvidia'], dtype=object)

In [282]:
train_limpio_4.Modelo_GPU.unique()

array(['Radeon R7 M445', 'HD Graphics 505', 'HD Graphics 520',
       'GeForce GTX 1070', 'GeForce GT 940MX', 'GeForce GTX 1050',
       'GeForce 940MX', 'GeForce GTX 1050 Ti', 'HD Graphics 405',
       'UHD Graphics 620', 'Radeon 530', 'HD Graphics 510',
       'HD Graphics 620', 'Quadro M2200M', 'GeForce MX150',
       'HD Graphics 400', 'Quadro M1200', 'GeForce 920MX',
       'HD Graphics 500', 'Radeon 520', 'Radeon R5 M330',
       'Radeon R5 M420X', 'HD Graphics', 'HD Graphics 630',
       'Radeon R5 M430', 'Iris Pro Graphics', 'GeForce GTX 940MX',
       'HD Graphics 515', 'GeForce GTX 1080', 'Radeon R2 Graphics',
       'GeForce 150MX', 'Radeon R5', 'GeForce GTX 1050Ti',
       'Radeon R9 M385', 'GeForce 930MX', 'GeForce GTX 1060',
       'GeForce GTX 950M', 'GeForce GTX 960M', 'HD Graphics 615',
       'FirePro W4190M', 'Radeon R4', 'Quadro M500M', 'Radeon RX 580',
       'GeForce GTX 970M', 'Radeon R7 M365X', 'Radeon R2',
       'Quadro M1000M', 'GeForce GTX 965M', 'GeForce MX

In [283]:
def separar_modelo_tipo(gpu):
    # Definir las palabras clave para los tipos conocidos
    tipos_conocidos = [
        'AMD R17M-M1-70', 'Nvidia GTX 980 SLI', 'AMD R4 Graphics', 'Radeon', 
        'HD Graphics', 'UHD Graphics', 'GeForce', 'Quadro', 'Iris', 'FirePro', 'Graphics',"Nvidia"
    ]
    
    # Buscar el tipo en la cadena
    tipo = 'Unknown'
    for tipo_conocido in tipos_conocidos:
        if gpu.startswith(tipo_conocido):
            tipo = tipo_conocido
            break
    
    # Extraer el modelo eliminando el tipo encontrado
    modelo = gpu.replace(tipo, '').strip()
    
    return pd.Series([tipo, modelo], index=['Tipo', 'Modelo'])

In [284]:
train_limpio_4[['Tipo_GPU', 'Modelo_GPU']] = train_limpio_4['Modelo_GPU'].apply(separar_modelo_tipo)

In [285]:
train_limpio_4.loc[train_limpio_4.Tipo_GPU == "Unknown"]

,laptop_ID,Company,Gpu,Price_euros,Pantalla,Marca_Procesador,Modelo_Procesador,Categoria_Peso,Velocidad_Procesador_GHz,S.O,Ram_GB,Tipo_Portatil,Nombre_Producto,Total_Almacenamiento,Marca_GPU,Modelo_GPU,Tipo_GPU
id,,,,,,,,,,,,,,,,,
693,182,Lenovo,AMD R17M-M1-70,813.0,FHD,Intel,Core i5,Medio,2.5,Windows 10,8,Notebook,V310-15IKB,2560.0,AMD,R17M-M1-70,Unknown
320,181,MSI,Nvidia GTX 980 SLI,2799.0,FHD,Intel,Core i7,Pesado,2.9,Windows 10,32,Gaming,GT80S 6QF-074US,4096.0,Nvidia,GTX 980 SLI,Unknown
1008,85,Lenovo,AMD R4 Graphics,399.0,FHD,AMD,A6-Series,Medio,2.5,Windows 10,4,Notebook,IdeaPad 320-15AST,512.0,AMD,R4 Graphics,Unknown


In [286]:
#Las anteriores las clasifica Unknown vamos a modificarlas. Sabemos la primera es Radeon, la segunda GeForce y la tercera Radeon.

In [287]:
train_limpio_4.loc[train_limpio_4.laptop_ID == 182, "Tipo_GPU"] = "Radeon"
train_limpio_4.loc[train_limpio_4.laptop_ID == 181, "Tipo_GPU"] = "Geforce"
train_limpio_4.loc[train_limpio_4.laptop_ID == 85, "Tipo_GPU"] = "Radeon"

In [288]:
train_limpio_4.head(3)

,laptop_ID,Company,Gpu,Price_euros,Pantalla,Marca_Procesador,Modelo_Procesador,Categoria_Peso,Velocidad_Procesador_GHz,S.O,Ram_GB,Tipo_Portatil,Nombre_Producto,Total_Almacenamiento,Marca_GPU,Modelo_GPU,Tipo_GPU
id,,,,,,,,,,,,,,,,,
268,1002,Dell,AMD Radeon R7 M445,749.0,WXGA,Intel,Core i7,Medio,2.7,Windows 10,8,Notebook,Inspiron 5567,2048.0,AMD,R7 M445,Radeon
347,867,Asus,Intel HD Graphics 505,449.0,WXGA,Intel,Pentium Quad Core,Medio,1.1,Windows 10,4,Notebook,X541NA,2048.0,Intel,505,HD Graphics
353,966,Toshiba,Intel HD Graphics 520,1460.0,FHD,Intel,Core i5,Ligero,2.3,Windows 7,8,Notebook,Portege Z30-C-1CW,1024.0,Intel,520,HD Graphics


In [289]:
train_limpio_4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 268 to 1179
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   laptop_ID                 912 non-null    int64  
 1   Company                   912 non-null    object 
 2   Gpu                       912 non-null    object 
 3   Price_euros               912 non-null    float64
 4   Pantalla                  912 non-null    object 
 5   Marca_Procesador          912 non-null    object 
 6   Modelo_Procesador         912 non-null    object 
 7   Categoria_Peso            912 non-null    object 
 8   Velocidad_Procesador_GHz  912 non-null    float64
 9   S.O                       912 non-null    object 
 10  Ram_GB                    912 non-null    int32  
 11  Tipo_Portatil             912 non-null    object 
 12  Nombre_Producto           912 non-null    object 
 13  Total_Almacenamiento      912 non-null    float64
 14  Marca_GPU   

In [290]:
train_limpio_5 = train_limpio_4.drop(columns=["Gpu"])
train_limpio_5.to_csv(r"D:\DATASCIENCE\Repositorios\Kaggle_Desafio\Train\TrainLimpio5.csv", index=True)

In [291]:
train_limpio_5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 268 to 1179
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   laptop_ID                 912 non-null    int64  
 1   Company                   912 non-null    object 
 2   Price_euros               912 non-null    float64
 3   Pantalla                  912 non-null    object 
 4   Marca_Procesador          912 non-null    object 
 5   Modelo_Procesador         912 non-null    object 
 6   Categoria_Peso            912 non-null    object 
 7   Velocidad_Procesador_GHz  912 non-null    float64
 8   S.O                       912 non-null    object 
 9   Ram_GB                    912 non-null    int32  
 10  Tipo_Portatil             912 non-null    object 
 11  Nombre_Producto           912 non-null    object 
 12  Total_Almacenamiento      912 non-null    float64
 13  Marca_GPU                 912 non-null    object 
 14  Modelo_GPU  

In [292]:
train_limpio_5.columns.tolist()

['laptop_ID',
 'Company',
 'Price_euros',
 'Pantalla',
 'Marca_Procesador',
 'Modelo_Procesador',
 'Categoria_Peso',
 'Velocidad_Procesador_GHz',
 'S.O',
 'Ram_GB',
 'Tipo_Portatil',
 'Nombre_Producto',
 'Total_Almacenamiento',
 'Marca_GPU',
 'Modelo_GPU',
 'Tipo_GPU']

In [293]:
train_limpio_5.Modelo_GPU.unique()

array(['R7 M445', '505', '520', 'GTX 1070', 'GT 940MX', 'GTX 1050',
       '940MX', 'GTX 1050 Ti', '405', '620', '530', '510', 'M2200M',
       'MX150', '400', 'M1200', '920MX', '500', 'R5 M330', 'R5 M420X', '',
       '630', 'R5 M430', 'Pro Graphics', 'GTX 940MX', '515', 'GTX 1080',
       'R2 Graphics', '150MX', 'R5', 'GTX 1050Ti', 'R9 M385', '930MX',
       'GTX 1060', 'GTX 950M', 'GTX 960M', '615', 'W4190M', 'R4', 'M500M',
       'RX 580', 'GTX 970M', 'R7 M365X', 'R2', 'M1000M', 'GTX 965M',
       'MX130', 'R5 M420', 'R7 M440', '6000', 'GTX 980M', 'M2000M',
       'Plus Graphics 650', 'RX 550', 'Plus Graphics 640', '5300',
       'R5 520', '960M', '930M', '3000M', 'GTX 1070M', 'M620M', 'R5 M315',
       'Pro 555', 'M620', 'GTX 960<U+039C>', 'M2200', '920M', 'M3000M',
       'R17M-M1-70', 'GTX 940M', 'R7 M460', 'R7 M360', 'R4 Graphics',
       'Pro 455', 'RX 560', 'Graphics 550', 'GTX 980 SLI', 'GTX1050 Ti',
       'Graphics 540', 'W6150M', 'R7 Graphics', 'R7', 'M520M'],
      dtype

In [294]:
train_limpio_5.head(10)

,laptop_ID,Company,Price_euros,Pantalla,Marca_Procesador,Modelo_Procesador,Categoria_Peso,Velocidad_Procesador_GHz,S.O,Ram_GB,Tipo_Portatil,Nombre_Producto,Total_Almacenamiento,Marca_GPU,Modelo_GPU,Tipo_GPU
id,,,,,,,,,,,,,,,,
268,1002,Dell,749.00,WXGA,Intel,Core i7,Medio,2.7,Windows 10,8,Notebook,Inspiron 5567,2048.0,AMD,R7 M445,Radeon
347,867,Asus,449.00,WXGA,Intel,Pentium Quad Core,Medio,1.1,Windows 10,4,Notebook,X541NA,2048.0,Intel,505,HD Graphics
353,966,Toshiba,1460.00,FHD,Intel,Core i5,Ligero,2.3,Windows 7,8,Notebook,Portege Z30-C-1CW,1024.0,Intel,520,HD Graphics
578,767,Dell,2868.99,4K,Intel,Core i7,Pesado,2.8,Windows 10,16,Gaming,Alienware 17,3072.0,Nvidia,GTX 1070,GeForce
42,1241,Dell,1713.37,FHD,Intel,Core i5,Ligero,2.4,Windows 7,8,Ultrabook,Latitude E7270,1024.0,Intel,520,HD Graphics
867,376,Asus,1099.00,FHD,Intel,Core i7,Medio,2.7,Windows 10,12,Convertible,ZenBook Flip,6144.0,Nvidia,GT 940MX,GeForce
1136,60,Asus,415.00,FHD,Intel,Core i3,Medio,2.0,Linux,4,Notebook,X541UA-DM1897,1024.0,Intel,520,HD Graphics
609,1157,MSI,1142.80,FHD,Intel,Core i7,Medio,2.8,Windows 10,8,Gaming,GP62M 7RDX,2560.0,Nvidia,GTX 1050,GeForce
474,110,Asus,1193.00,FHD,Intel,Core i7,Ligero,2.7,Windows 10,8,Ultrabook,UX430UQ-GV209R,1024.0,Nvidia,940MX,GeForce


In [295]:
train_limpio_5.Tipo_GPU.unique()

array(['Radeon', 'HD Graphics', 'GeForce', 'UHD Graphics', 'Quadro',
       'Iris', 'FirePro', 'Geforce', 'Graphics'], dtype=object)

In [296]:
train_limpio_5['Tipo_GPU'] = train_limpio_5['Tipo_GPU'].replace({'Geforce': 'GeForce', 'HD Graphics': 'Graphics'})


In [297]:
orden_columnas = ['laptop_ID',
 'Company',
 'Nombre_Producto',
 'Tipo_Portatil',
 'S.O',
 'Marca_Procesador',
 'Modelo_Procesador',
 'Velocidad_Procesador_GHz',
 'Ram_GB',
 'Total_Almacenamiento',
 'Marca_GPU',
 'Modelo_GPU',
 'Tipo_GPU',
 'Categoria_Peso',
 'Pantalla',
 'Price_euros']

In [298]:
train_limpio_5 = train_limpio_5[orden_columnas]

In [299]:
train_limpio_5.Modelo_GPU.value_counts()

Modelo_GPU
620             246
520             149
GTX 1050         43
GTX 1060         31
400              28
               ... 
R4                1
R9 M385           1
Pro Graphics      1
M2200M            1
M520M             1
Name: count, Length: 84, dtype: int64

###TRANSFORMACION DE CATEGORICAS A NÚMERICAS

In [430]:
#Vamos a quedarnos con las columnas que creemos que no van a tener utilidad

train_final = train_limpio_5.drop(columns=["laptop_ID"])

In [431]:
train_final = train_final.applymap(lambda x: x.lower() if isinstance(x, str) else x)

C:\Users\raul_\AppData\Local\Temp\ipykernel_8364\1420150622.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_final = train_final.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [302]:

train_final.to_csv(r"D:\DATASCIENCE\Repositorios\Kaggle_Desafio\Train\Trainfinal.csv", index=True)

In [432]:
train_final.head(5)

,Company,Nombre_Producto,Tipo_Portatil,S.O,Marca_Procesador,Modelo_Procesador,Velocidad_Procesador_GHz,Ram_GB,Total_Almacenamiento,Marca_GPU,Modelo_GPU,Tipo_GPU,Categoria_Peso,Pantalla,Price_euros
id,,,,,,,,,,,,,,,
268,dell,inspiron 5567,notebook,windows 10,intel,core i7,2.7,8,2048.0,amd,r7 m445,radeon,medio,wxga,749.00
347,asus,x541na,notebook,windows 10,intel,pentium quad core,1.1,4,2048.0,intel,505,graphics,medio,wxga,449.00
353,toshiba,portege z30-c-1cw,notebook,windows 7,intel,core i5,2.3,8,1024.0,intel,520,graphics,ligero,fhd,1460.00
578,dell,alienware 17,gaming,windows 10,intel,core i7,2.8,16,3072.0,nvidia,gtx 1070,geforce,pesado,4k,2868.99
42,dell,latitude e7270,ultrabook,windows 7,intel,core i5,2.4,8,1024.0,intel,520,graphics,ligero,fhd,1713.37


#Separamos las categoricas

categorical_columns = ["Company", "Tipo_Portatil","S.O","Marca_Procesador","Modelo_Procesador","Marca_GPU","Modelo_GPU","Tipo_GPU","Categoria_Peso","Pantalla"]


#Le pasamos el label encoding para transformar las categoricas a númericas, IMPORTANTE no hemos metido la columna de Price_euros que ahora la dividiremos

label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    train_final[column] = le.fit_transform(train_final[column])
    label_encoders[column] = le

In [304]:
train_final

,Company,Tipo_Portatil,S.O,Marca_Procesador,Modelo_Procesador,Velocidad_Procesador_GHz,Ram_GB,Total_Almacenamiento,Marca_GPU,Modelo_GPU,Tipo_GPU,Categoria_Peso,Pantalla,Price_euros
id,,,,,,,,,,,,,,
268,dell,notebook,windows 10,intel,core i7,2.7,8,2048.0,amd,r7 m445,radeon,medio,wxga,749.00
347,asus,notebook,windows 10,intel,pentium quad core,1.1,4,2048.0,intel,505,graphics,medio,wxga,449.00
353,toshiba,notebook,windows 7,intel,core i5,2.3,8,1024.0,intel,520,graphics,ligero,fhd,1460.00
578,dell,gaming,windows 10,intel,core i7,2.8,16,3072.0,nvidia,gtx 1070,geforce,pesado,4k,2868.99
42,dell,ultrabook,windows 7,intel,core i5,2.4,8,1024.0,intel,520,graphics,ligero,fhd,1713.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,acer,convertible,chrome os,intel,celeron dual core,1.6,4,32.0,intel,400,graphics,ligero,wxga,389.00
716,toshiba,notebook,windows 10,intel,core i5,2.3,4,512.0,intel,520,graphics,ligero,fhd,1195.00
517,acer,notebook,windows 10,amd,a8-series,2.2,8,2048.0,amd,r5,radeon,medio,wxga,449.00


-----------------------------------------------------------------------------------------------------------------

## Pasos que si o si debéis realizar para poder participar en la competición:

### 1. Definir X e y

### 2. Dividir X_train, X_test, y_train, y_test

In [305]:
aaaaaaaaa

NameError: name 'aaaaaaaaa' is not defined

###RANDOM FOREST

In [398]:

# Identificar características categóricas y numéricas
categorical_features = ['Company', 'Tipo_Portatil', 'S.O', 'Marca_Procesador', 'Modelo_Procesador', 
                        'Marca_GPU', 'Modelo_GPU', 'Tipo_GPU', 'Categoria_Peso', 'Pantalla']
numerical_features = ['Velocidad_Procesador_GHz', 'Ram_GB', 'Total_Almacenamiento']




# Preprocesamiento para características categóricas y numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

model_RANDOM = RandomForestRegressor(random_state=42, )

# Crear el pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model_RANDOM)
])


param_grid_RANDOM=  {
    'model__n_estimators': [100, 200, 300],
    'model__max_features': ['auto', 'sqrt', 'log2'],
    'model__max_depth': [None, 10, 20, 30],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4],
    'model__bootstrap': [True, False]
}
# Definir el modelo
grid_search_RANDOM = GridSearchCV(pipeline, param_grid_RANDOM, cv=5, n_jobs=-1, scoring='neg_mean_absolute_error')


# Dividir el dataset en conjuntos de entrenamiento y prueba
X = train_final.drop(columns=['Price_euros'])
y = train_final['Price_euros']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
grid_search_RANDOM.fit(X_train, y_train)

best_params_RANDOM = grid_search_RANDOM.best_params_
best_model_RANDOM = grid_search_RANDOM.best_estimator_

y_pred_RANDOM = best_model_RANDOM.predict(X_test)
mae_RANDOM = mean_absolute_error(y_test, y_pred_RANDOM)


c:\Users\raul_\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1080 fits failed out of a total of 3240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
639 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\raul_\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\raul_\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\raul_\A

In [399]:
best_params_RANDOM

{'model__bootstrap': False,
 'model__max_depth': 20,
 'model__max_features': 'sqrt',
 'model__min_samples_leaf': 1,
 'model__min_samples_split': 2,
 'model__n_estimators': 300}

In [400]:
mae_RANDOM

171.25206122054007

### 3. Crear y entrenar el modelo


#GRADIENT BOOSTING

In [436]:
# Identificar características categóricas y numéricas
categorical_features = ['Company', 'Tipo_Portatil', 'S.O', 'Marca_Procesador', 'Modelo_Procesador', 
                        'Marca_GPU', 'Modelo_GPU', 'Tipo_GPU', 'Categoria_Peso', 'Pantalla']
numerical_features = ['Velocidad_Procesador_GHz', 'Ram_GB', 'Total_Almacenamiento']




# Preprocesamiento para características categóricas y numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

model_gradient = GradientBoostingRegressor(random_state=42)

# Crear el pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model_gradient)
])

param_grid_gradient =  {
    'model__learning_rate': [0.01, 0.1],
    'model__loss': ['squared_error','huber'],
    'model__max_depth': [7, 9],
    'model__max_features': ['sqrt', 'log2'],
    'model__min_samples_leaf': [2, 4],
    'model__min_samples_split': [2, 5],
    'model__n_estimators': [ 200, 300,400],
    'model__subsample': [0.8, 1.0]
}
# Definir el modelo
grid_search_gradient = GridSearchCV(pipeline, param_grid_gradient, cv=5, n_jobs=-1, scoring='neg_mean_absolute_error')


# Dividir el dataset en conjuntos de entrenamiento y prueba
X = train_final.drop(columns=['Price_euros'])
y = train_final['Price_euros']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
grid_search_gradient.fit(X_train, y_train)

best_params_gradient = grid_search_gradient.best_params_
best_model_gradient = grid_search_gradient.best_estimator_

y_pred_gradient = best_model_gradient.predict(X_test)
mae_gradient = mean_absolute_error(y_test, y_pred_gradient)

In [441]:
best_params_gradient

{'model__learning_rate': 0.1,
 'model__loss': 'huber',
 'model__max_depth': 9,
 'model__max_features': 'log2',
 'model__min_samples_leaf': 2,
 'model__min_samples_split': 5,
 'model__n_estimators': 300,
 'model__subsample': 0.8}

In [438]:
mae_gradient

168.02494458837137

#CATBOOST

In [309]:
# Identificar características categóricas y numéricas
categorical_features = ['Company', 'Tipo_Portatil', 'S.O', 'Marca_Procesador', 'Modelo_Procesador', 
                        'Marca_GPU', 'Modelo_GPU', 'Tipo_GPU', 'Categoria_Peso', 'Pantalla']
numerical_features = ['Velocidad_Procesador_GHz', 'Ram_GB', 'Total_Almacenamiento']




# Preprocesamiento para características categóricas y numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

model_CAT = CatBoostRegressor(random_state=42,verbose=0 )

# Crear el pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model_CAT)
])


param_grid_CAT=   {
    'model__iterations': [100, 200, 500],
    'model__depth': [4, 6, 10],
    'model__learning_rate': [0.01, 0.1, 0.3],
    'model__l2_leaf_reg': [1, 3, 5]
}
# Definir el modelo
grid_search_CAT = GridSearchCV(pipeline, param_grid_CAT, cv=5, n_jobs=-1, scoring='neg_mean_absolute_error')


# Dividir el dataset en conjuntos de entrenamiento y prueba
X = train_final.drop(columns=['Price_euros'])
y = train_final['Price_euros']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
grid_search_CAT.fit(X_train, y_train)

best_params_CAT = grid_search_CAT.best_params_
best_model_CAT = grid_search_CAT.best_estimator_

y_pred_CAT = best_model_CAT.predict(X_test)
mae_CAT = mean_absolute_error(y_test, y_pred_CAT)


In [313]:
X

,Company,Tipo_Portatil,S.O,Marca_Procesador,Modelo_Procesador,Velocidad_Procesador_GHz,Ram_GB,Total_Almacenamiento,Marca_GPU,Modelo_GPU,Tipo_GPU,Categoria_Peso,Pantalla
id,,,,,,,,,,,,,
268,dell,notebook,windows 10,intel,core i7,2.7,8,2048.0,amd,r7 m445,radeon,medio,wxga
347,asus,notebook,windows 10,intel,pentium quad core,1.1,4,2048.0,intel,505,graphics,medio,wxga
353,toshiba,notebook,windows 7,intel,core i5,2.3,8,1024.0,intel,520,graphics,ligero,fhd
578,dell,gaming,windows 10,intel,core i7,2.8,16,3072.0,nvidia,gtx 1070,geforce,pesado,4k
42,dell,ultrabook,windows 7,intel,core i5,2.4,8,1024.0,intel,520,graphics,ligero,fhd
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,acer,convertible,chrome os,intel,celeron dual core,1.6,4,32.0,intel,400,graphics,ligero,wxga
716,toshiba,notebook,windows 10,intel,core i5,2.3,4,512.0,intel,520,graphics,ligero,fhd
517,acer,notebook,windows 10,amd,a8-series,2.2,8,2048.0,amd,r5,radeon,medio,wxga


In [310]:
best_params_CAT

{'model__depth': 6,
 'model__iterations': 500,
 'model__l2_leaf_reg': 1,
 'model__learning_rate': 0.1}

In [311]:
mae_CAT

182.38702646581527

##XGBOOST

In [ ]:
XGB= xgb.XGBRegressor(objective='reg:squarederror')


In [444]:
# Identificar características categóricas y numéricas
categorical_features = ['Company', 'Tipo_Portatil', 'S.O', 'Marca_Procesador', 'Modelo_Procesador', 
                        'Marca_GPU', 'Modelo_GPU', 'Tipo_GPU', 'Categoria_Peso', 'Pantalla']
numerical_features = ['Velocidad_Procesador_GHz', 'Ram_GB', 'Total_Almacenamiento']




# Preprocesamiento para características categóricas y numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

model_xgb = XGBRegressor(objective="reg:squarederror",random_state=42)

# Crear el pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model_xgb)
])


param_grid_xgb=  {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth': [3, 5, 7],
    'model__learning_rate': [0.01, 0.1, 0.2],
    'model__subsample': [0.5, 0.7, 0.8],
    'model__colsample_bytree': [0.5,0.7],
    'model__gamma': [0, 0.1, 0.125],
    'model__reg_alpha': [0, 0.01, 0.05],
    'model__reg_lambda': [1, 1.25]
}
# Definir el modelo
grid_search_xgb = GridSearchCV(pipeline, param_grid_xgb, cv=5, n_jobs=-1, scoring='neg_mean_absolute_error')


# Dividir el dataset en conjuntos de entrenamiento y prueba
X = train_final.drop(columns=['Price_euros'])
y = train_final['Price_euros']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
grid_search_xgb.fit(X_train, y_train)

best_params_xgb = grid_search_xgb.best_params_
best_model_xgb = grid_search_xgb.best_estimator_

y_pred_xgb = best_model_xgb.predict(X_test)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)


In [445]:
mae_xgb

166.39975131008794

In [446]:
best_params_xgb

{'model__colsample_bytree': 0.5,
 'model__gamma': 0.125,
 'model__learning_rate': 0.1,
 'model__max_depth': 7,
 'model__n_estimators': 200,
 'model__reg_alpha': 0,
 'model__reg_lambda': 1.25,
 'model__subsample': 0.5}

SVR

In [419]:
# Identificar características categóricas y numéricas
categorical_features = ['Company', 'Tipo_Portatil', 'S.O', 'Marca_Procesador', 'Modelo_Procesador', 
                        'Marca_GPU', 'Modelo_GPU', 'Tipo_GPU', 'Categoria_Peso', 'Pantalla']
numerical_features = ['Velocidad_Procesador_GHz', 'Ram_GB', 'Total_Almacenamiento']




# Preprocesamiento para características categóricas y numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

model_SVR = SVR()

# Crear el pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model_SVR)
])


param_grid_SVR=  {
    'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'model__C': [0.1, 1, 10, 100],
    'model__gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
    'model__degree': [2, 3, 4,5],
    'model__epsilon': [0.1, 0.2, 0.5, 0.3,0.7],
    'model__shrinking': [True, False]
}
# Definir el modelo
grid_search_SVR = GridSearchCV(pipeline, param_grid_SVR, cv=5, n_jobs=-1, scoring='neg_mean_absolute_error')


# Dividir el dataset en conjuntos de entrenamiento y prueba
X = train_final.drop(columns=['Price_euros'])
y = train_final['Price_euros']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
grid_search_SVR.fit(X_train, y_train)

best_params_SVR = grid_search_SVR.best_params_
best_model_SVR = grid_search_SVR.best_estimator_

y_pred_SVR = best_model_SVR.predict(X_test)
mae_SVR = mean_absolute_error(y_test, y_pred_SVR)

In [420]:
best_params_SVR

{'model__C': 1,
 'model__degree': 3,
 'model__epsilon': 0.1,
 'model__gamma': 1,
 'model__kernel': 'poly',
 'model__shrinking': False}

In [421]:
mae_SVR

186.8332708912883

### 4. Sacar métricas, valorar el modelo 

Recuerda que en la competición se va a evaluar con la métrica de MAE.

In [ ]:
## RIDGE MAE
print("Mejores hiperparámetros:", grid_search_Ridge.best_params_)
#Hacemos el MAE
mae_Ridge = mean_absolute_error(y_test, y_pred_Ridge)
print("Mean Absolute Error (MAE):", mae_Ridge)

Mejores hiperparámetros: {'alpha': 10.0, 'solver': 'saga'}
Mean Absolute Error (MAE): 292.91520216289376


In [ ]:
##RAMDONFOREST MAE
print("Mejores hiperparámetros:", grid_search_RandomForest.best_params_)
#Hacemos el MAE
mae_RandomForest = mean_absolute_error(y_test, y_pred_RandomForest)
print("Mean Absolute Error (MAE):", mae_RandomForest)

Mejores hiperparámetros: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 400}
Mean Absolute Error (MAE): 171.73554565433597


In [ ]:
##GRADIANT BOOST
print("Mejores hiperparámetros:", grid_search_Boost.best_params_)
#Hacemos el MAE
mae_Boost = mean_absolute_error(y_test, y_pred_Boost)
print("Mean Absolute Error (MAE):", mae_Boost)



Mejores hiperparámetros: {'learning_rate': 0.1, 'loss': 'squared_error', 'max_depth': 9, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 12, 'n_estimators': 100, 'subsample': 1.0}
Mean Absolute Error (MAE): 170.7959232586437


In [ ]:
##CAT BOOST
print("Mejores hiperparámetros:", grid_search_Cat.best_params_)
#Hacemos el MAE
mae_Cat = mean_absolute_error(y_test, y_pred_Cat)
print("Mean Absolute Error (MAE):", mae_Cat)

Mejores hiperparámetros: {'depth': 4, 'iterations': 400, 'l2_leaf_reg': 1, 'learning_rate': 0.125}
Mean Absolute Error (MAE): 169.7644499498742


-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `test.csv` nos arriesgamos a que digamos, no salga lo esperado.

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `train.csv` al completo.


**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

### 2. Carga los datos de `test.csv` para predecir.

In [447]:
ruta = r"D:\DATASCIENCE\Repositorios\Kaggle_Desafio\Test\test.csv"
test_final = pd.read_csv(ruta, index_col="id")


test_final["Resolucion"] = test_final['ScreenResolution'].str.extract(patron)
test_final['Pantalla'] = test_final['Resolucion'].apply(clasificar_resolucion)
test_final['Marca_Procesador'] = test_final['Cpu'].apply(extract_brand)
test_final['Modelo_Procesador'] = test_final['Cpu'].apply(extract_model)
test_final['Velocidad_Procesador'] = test_final['Cpu'].apply(extract_speed)
test_final[['SSD', 'HDD', 'Flash Storage', 'Hybrid']] = test_final['Memory'].apply(lambda x: pd.Series(separar_discos(x)))
test_final['Peso_KG'] = test_final['Weight'].apply(extraer_numerico)
test_final = test_final.drop(columns=["Inches","Weight","ScreenResolution","Cpu","Memory","Resolucion"])
test_final['Categoria_Peso'] = test_final['Peso_KG'].apply(categorize_weight)
test_final['Velocidad_Procesador_GHz'] = test_final['Velocidad_Procesador'].str.extract(r'(\d+\.?\d*)').astype(float)
test_final['Ram_GB'] = test_final['Ram'].str.extract(r'(\d+)\s*').astype(int)
test_final["S.O"] = test_final["OpSys"]
test_final = test_final.drop(columns=["Ram","OpSys","Velocidad_Procesador","Peso_KG"])
test_final['Tipo_Portatil'] = test_final['TypeName'].str.replace('2 in 1', '').str.strip()
test_final['Nombre_Producto'] = test_final['Product'].apply(limpieza_producto)
test_final = test_final.drop(columns=["TypeName","Product"])
test_final['SSD'] = test_final['SSD'].apply(convert_storage)
test_final['HDD'] = test_final['HDD'].apply(convert_storage)
test_final['Flash Storage'] = test_final['Flash Storage'].apply(convert_storage)
test_final['Hybrid'] = test_final['Hybrid'].apply(convert_storage)
test_final['Total_Almacenamiento'] = (test_final['SSD'] * 4) + (test_final['Hybrid'] * 3) + (test_final['HDD'] * 2) + test_final['Flash Storage']
test_final = test_final.drop(columns=["SSD","HDD","Flash Storage","Hybrid"])
test_final[['Marca_GPU', 'Modelo_GPU']] = test_final['Gpu'].apply(separar_marca_modelo)
test_final[['Tipo_GPU', 'Modelo_GPU']] = test_final['Modelo_GPU'].apply(separar_modelo_tipo)
test_final = test_final.drop(columns=["Gpu"])
test_final['Tipo_GPU'] = test_final['Tipo_GPU'].replace({'Geforce': 'GeForce', 'HD Graphics': 'Graphics'})
test_finalorden_columnas_test = ['laptop_ID',
 'Company',
 'Nombre_Producto',
 'Tipo_Portatil',
 'S.O',
 'Marca_Procesador',
 'Modelo_Procesador',
 'Velocidad_Procesador_GHz',
 'Ram_GB',
 'Total_Almacenamiento',
 'Marca_GPU',
 'Modelo_GPU',
 'Tipo_GPU',
 'Categoria_Peso',
 'Pantalla'
]
test_final = test_final[test_finalorden_columnas_test]
test_final = test_final.drop(columns=["laptop_ID", "Nombre_Producto"])
test_final = test_final.applymap(lambda x: x.lower() if isinstance(x, str) else x)

C:\Users\raul_\AppData\Local\Temp\ipykernel_8364\2174353472.py:49: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_final = test_final.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [377]:
test_final

,Company,Tipo_Portatil,S.O,Marca_Procesador,Modelo_Procesador,Velocidad_Procesador_GHz,Ram_GB,Total_Almacenamiento,Marca_GPU,Modelo_GPU,Tipo_GPU,Categoria_Peso,Pantalla
id,,,,,,,,,,,,,
181,hp,ultrabook,windows 10,intel,core i7,2.7,16,2048.0,intel,620,graphics,ligero,4k
708,acer,notebook,windows 10,amd,a12-series,2.7,8,1024.0,amd,rx 540,radeon,medio,wxga
862,acer,notebook,linux,intel,core i3,2.0,4,1000.0,intel,520,graphics,medio,wxga
1064,hp,notebook,windows 7,intel,core i5,2.3,8,1024.0,intel,520,graphics,ligero,fhd
702,hp,notebook,windows 10,intel,core i5,2.5,8,1024.0,intel,620,graphics,ligero,fhd
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,lenovo,gaming,no os,intel,core i7,2.8,8,1024.0,nvidia,gtx 1050m,geforce,medio,fhd
524,lenovo,gaming,windows 10,intel,core i7,2.6,16,2048.0,nvidia,gtx 960,geforce,pesado,fhd
1015,hp,notebook,no os,intel,core i5,2.5,4,1000.0,intel,620,graphics,medio,wxga


**¿Por qué puede dar error?** 

IMPORTANTE: APLICAR A ESTOS DATOS LO MISMO QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 391 FILAS, SI O SI

**Entonces, si al cargar los datos de train usé `index_col=0` para que utilizara la primera columna del conjunto de datos como índice, ¿tendré que hacerlo también para el conjunto `test.csv`?**

In [ ]:
# ¿Qué opináis?
# Sí

![](https://www.tierraljelechu.com/web/img/wiki_up/1.996-SorpresaDto.-1-Red.jpg)

### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle. 

**¿Qué es lo que subirás a Kaggle?**

In [448]:
test_final.reset_index(inplace=True)

In [449]:
test_final["Price_euros"] = best_model_xgb.predict(test_final)


In [450]:
best_model_xgb.predict(test_final)

array([1676.0728 ,  678.5951 ,  389.5271 , 1482.4642 , 1234.3448 ,
       1229.3173 , 2803.3025 ,  690.4636 , 2736.7698 , 2067.2957 ,
       1436.1191 ,  740.5753 ,  290.4441 ,  556.9642 ,  410.07727,
       1268.0372 ,  610.9243 , 2003.0842 , 1390.3378 , 1582.7672 ,
       1444.9586 ,  998.5965 , 1711.0851 , 1296.5278 , 1882.6713 ,
        386.82657,  913.5341 ,  452.35025, 1085.8983 ,  498.42276,
        999.9508 , 1299.2175 , 1814.9135 ,  311.34265,  581.90765,
       1655.8405 ,  804.52295,  637.2767 , 2938.6519 ,  455.9611 ,
        886.57776,  877.21344, 1039.0363 , 1814.9135 ,  980.9418 ,
       1652.2146 ,  550.4398 , 4070.1296 ,  261.9598 , 1186.897  ,
       1136.0739 ,  963.90967,  967.56555,  343.38306, 1313.4744 ,
        706.5301 ,  993.89014,  242.38962, 1371.4523 , 1093.6659 ,
       1279.6537 , 1046.5995 ,  520.4449 , 1149.3428 , 1382.4467 ,
       1129.9327 ,  709.51245, 2323.6047 ,  484.4914 , 2537.0457 ,
        944.4473 , 2513.0166 , 2029.4751 ,  908.87524, 1401.69

In [451]:
test_final

,id,Company,Tipo_Portatil,S.O,Marca_Procesador,Modelo_Procesador,Velocidad_Procesador_GHz,Ram_GB,Total_Almacenamiento,Marca_GPU,Modelo_GPU,Tipo_GPU,Categoria_Peso,Pantalla,Price_euros
0,181,hp,ultrabook,windows 10,intel,core i7,2.7,16,2048.0,intel,620,graphics,ligero,4k,1676.072754
1,708,acer,notebook,windows 10,amd,a12-series,2.7,8,1024.0,amd,rx 540,radeon,medio,wxga,678.595093
2,862,acer,notebook,linux,intel,core i3,2.0,4,1000.0,intel,520,graphics,medio,wxga,389.527100
3,1064,hp,notebook,windows 7,intel,core i5,2.3,8,1024.0,intel,520,graphics,ligero,fhd,1482.464233
4,702,hp,notebook,windows 10,intel,core i5,2.5,8,1024.0,intel,620,graphics,ligero,fhd,1234.344849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,1281,lenovo,gaming,no os,intel,core i7,2.8,8,1024.0,nvidia,gtx 1050m,geforce,medio,fhd,985.740295
387,524,lenovo,gaming,windows 10,intel,core i7,2.6,16,2048.0,nvidia,gtx 960,geforce,pesado,fhd,1604.629517
388,1015,hp,notebook,no os,intel,core i5,2.5,4,1000.0,intel,620,graphics,medio,wxga,477.451141
389,1236,hp,notebook,windows 10,amd,e-series,1.5,4,1000.0,amd,r2,radeon,medio,fhd,382.080383


In [452]:
submision_xgb_2 = test_final[["id", "Price_euros"]]

In [453]:
submision_xgb_2

,id,Price_euros
0,181,1676.072754
1,708,678.595093
2,862,389.527100
3,1064,1482.464233
4,702,1234.344849
...,...,...
386,1281,985.740295
387,524,1604.629517
388,1015,477.451141
389,1236,382.080383


**¡PERO! Para subir a Kaggle la predicción, ésta tendrá que tener una forma específica y no valdrá otra.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [454]:
ruta = r"D:\DATASCIENCE\Repositorios\Kaggle_Desafio\Sample_Submission\sample_submission.csv"


In [455]:
sample = pd.read_csv(ruta) # Esta es mi ruta del archivo, la vuestra puede ser otra

In [456]:
sample.head()

,id,Price_euros
0,1014,752.0
1,845,499.0
2,1151,1747.0
3,1265,245.0
4,573,1179.0


In [ ]:
sample.shape

(391, 2)

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [457]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                submision_xgb_2.to_csv("submission_xgb_2.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("")


In [458]:
chequeator(submision_xgb_2) # submission es el nombre que le habríamos puesto a nuestro .csv con los valores que me salieron en la predicción

You're ready to submit!


NameError: name 'urllib' is not defined